# 05_mlops_cloud_deployment.ipynb

## Week 5: MLOps Essentials (Docker, Serving, CI/CD, Cloud, Monitoring)

### Notebook Overview
Welcome to **Week 5**, where you'll bridge the gap between model training and **production deployment**. By the end of this notebook, you will:
1. **Containerize** your ML model using Docker.
2. Create a **model serving** API (e.g., with FastAPI or Flask).
3. Set up a **CI/CD pipeline** to automatically build and test your Docker images.
4. **Deploy** the container to a **cloud** environment (AWS, GCP, or Azure).
5. Implement **monitoring & logging** to observe your model’s performance in production.

**Industry Context**: MLOps is critical at scale—FAANG and other companies use these practices to ensure reliable, reproducible, and maintainable machine learning services. Mastering these skills makes you stand out as a true AI/ML Engineer.

#### ADHD-Friendly Notes
- We’ve broken tasks down **day-by-day**.
- Each day’s tasks have **checklists** and short descriptions, so you can tackle them in small, **manageable chunks**.
- **Reward yourself** after completing each day’s tasks (short break, favorite snack, or relaxing activity). This helps sustain focus.

---
## 1. Monday: Docker Basics & Packaging Your Model

### Topics:
- **Docker** fundamentals: Dockerfile, building images, running containers.
- **Packaging** a simple ML model.

### Why This Matters
Docker allows you to ship your model and environment together, ensuring consistency across dev, test, and prod environments.

### Notebook Tasks:
1. **Install Docker** (if not already) and confirm it’s running (`docker --version`).
2. Write a **Dockerfile** that:
   - Uses a Python base image.
   - Installs required libraries (`scikit-learn`, `pandas`, etc.).
   - Copies your model/code into the container.
   - Specifies a command to run your script.
3. **Build & Run** the Docker image locally.

#### ADHD Tip
- Check off each step as you go. For example:
  1. Docker installed ✓
  2. Dockerfile created ✓
  3. Image built and tested ✓
- Use a sticky note or quick to-do app for micro-tracking.


### Example Dockerfile (Skeleton)
```Dockerfile
FROM python:3.9-slim

# Create app directory
WORKDIR /app

# Copy requirements and install
COPY requirements.txt requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy your code into the container
COPY . .

# Command to run your script
CMD ["python", "run_model.py"]
```

1. Create a file named `Dockerfile` in your project.
2. Create a `requirements.txt` listing dependencies (e.g., scikit-learn==1.0, fastapi==0.85, etc.).
3. In the terminal, run:
```
docker build -t my_ml_model:latest .
docker run -p 8080:8080 my_ml_model:latest
```

**Your Observations**:
- *(Document any issues encountered with dependencies. Note how you resolved them.)*

## 2. Tuesday: Model Serving with FastAPI (or Flask)

### Topics:
- Creating a **REST API** for inference.
- Exposing an endpoint (e.g., `/predict`) that accepts JSON input.

### Why This Matters
Clients or other services typically call a REST endpoint to get predictions. This is the standard approach in modern microservice architectures.

### Notebook Tasks:
1. **Install** FastAPI (e.g., `pip install fastapi uvicorn`).
2. Create a simple server script (`main.py`) that:
   - Loads a pre-trained model (e.g., scikit-learn .pkl file or a PyTorch model).
   - Defines a `/predict` endpoint that takes input data.
   - Returns predictions.
3. **Test** locally via `curl` or a small Python client.

#### ADHD Tip
- Write down a short code snippet for the `/predict` endpoint.
- Test in small increments (e.g., start the server, send one request, confirm the response, repeat).


### Example FastAPI Skeleton
```python
from fastapi import FastAPI
from pydantic import BaseModel
import joblib

app = FastAPI()

# Define input schema
class PredictRequest(BaseModel):
    feature1: float
    feature2: float

# Load your model
model = joblib.load("./my_model.pkl")

@app.post("/predict")
def predict(data: PredictRequest):
    # Convert input to the right shape
    X = [[data.feature1, data.feature2]]
    prediction = model.predict(X)
    return {"prediction": prediction[0]}
```

You’d typically run this with `uvicorn main:app --host 0.0.0.0 --port 8080`.

**Your Observations**:
- *(Write any notes about how you tested this or how you’d handle errors.)*

## 3. Wednesday: CI/CD Pipeline (GitHub Actions or Similar)

### Topics:
- **Continuous Integration**: Automated tests whenever you push code.
- **Continuous Delivery/Deployment**: Building and pushing Docker images automatically.

### Why This Matters
CI/CD ensures your code is always tested, your Docker image is consistently built, and your ML service can be deployed with minimal manual intervention.

### Notebook Tasks:
1. Create a **GitHub Actions** workflow file (e.g., `.github/workflows/docker-build.yml`).
2. Make it **build** your Docker image on every push.
3. Optionally run **unit tests** (like a small `test_model.py`).
4. Configure it to **push** the image to Docker Hub or GitHub Container Registry.

#### ADHD Tip
- Keep your CI steps minimal at first (just a Docker build and basic lint/test).
- Add complexity gradually (e.g., integration tests) once you have the basics down.


### Example GitHub Actions Workflow (Skeleton)
```yaml
name: CI/CD Pipeline

on:
  push:
    branches: ["main"]

jobs:
  build-and-push:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v2

      - name: Set up QEMU
        uses: docker/setup-qemu-action@v2

      - name: Set up Docker Buildx
        uses: docker/setup-buildx-action@v2

      - name: Build and push Docker image
        uses: docker/build-push-action@v2
        with:
          context: .
          file: ./Dockerfile
          push: true
          tags: your-dockerhub-username/your-repo:latest
          # Optionally specify more tags or versions
```

**Your Observations**:
- *(Note how you had to store Docker Hub credentials as GitHub secrets. Document each step to avoid confusion!)*

## 4. Thursday: Cloud Deployment

### Topics:
- Deploying to **AWS** (ECS, EKS, or Elastic Beanstalk), or **GCP** (Cloud Run), or Azure.
- Setting up your container to run on a managed service.
- Exposing a public endpoint.

### Why This Matters
Production ML systems need to be publicly accessible (or accessible within a company’s network). Cloud deployment is the standard approach to scale and manage.

### Notebook Tasks:
1. Choose **one cloud** platform (AWS, GCP, or Azure) to keep it simple.
2. Push your Docker image to a registry (AWS ECR, GCP GCR, or Docker Hub if you prefer).
3. Use a managed service to **run** your container (e.g., AWS ECS or GCP Cloud Run).
4. Validate you can call the **`/predict`** endpoint from outside.

#### ADHD Tip
- Focus on just **one** method of deployment. Don’t get lost in the variety of options.
- Write down each step (1. create a service, 2. configure CPU/memory, 3. open port 8080, etc.). Check them off as you succeed.


### Example: AWS ECS Fargate (Short Overview)
1. Create an **ECR repository**.
2. Use `docker login` to authenticate with ECR.
3. Tag and **push** your local image to ECR.
4. In the AWS console, create an **ECS cluster** (Fargate), define a **task** that references your ECR image.
5. Expose port 8080, set up a security group to allow inbound traffic.
6. **Run** the task, get the public IP or load balancer URL.
7. Test with `curl http://<YOUR_ECS_SERVICE_URL>:8080/predict`.

**Your Observations**:
- *(Describe any difficulties or clarifications needed. For instance, IAM roles, region settings, or VPC configurations.)*

## 5. Friday: Monitoring & Logging

### Topics:
- **Logging** (stdout, CloudWatch, Stackdriver, etc.).
- **Basic metrics** (request counts, latency, error rates).
- Potential introduction to **model performance monitoring** (concept drift, data drift, etc.).

### Why This Matters
Monitoring ensures you know how your model performs and if anything breaks. In production, unseen issues (e.g., changes in data distribution) can degrade performance silently.

### Notebook Tasks:
1. Enable **logs** on your cloud service or container orchestrator (e.g., CloudWatch logs for ECS, Stackdriver for GCP).
2. Print logs for each request in your FastAPI/Flask code.
3. (Optional) Store prediction inputs/outputs somewhere (if privacy/compliance rules allow) for later analysis.
4. Summarize how you’d detect data drift or degrade in model performance over time.

#### ADHD Tip
- Write down a small function or snippet that logs the request time, client IP, request payload, etc.
- Keep it simple. Detailed logging frameworks can be added later.


### Example Logging Snippet (FastAPI)
```python
import logging
from fastapi import FastAPI, Request

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

@app.middleware("http")
async def log_requests(request: Request, call_next):
    logger.info(f"Incoming request {request.url} from {request.client}")
    response = await call_next(request)
    logger.info(f"Response status {response.status_code}")
    return response
```

**Your Observations**:
- *(Document how you see these logs in CloudWatch or local logs. Are they easy to read?)*

## Weekend: End-to-End MLOps Mini-Project

**Objective**: Combine the entire pipeline into a small, but complete example.
1. **Train or reuse** an ML model (could be a scikit-learn model from earlier weeks).
2. **Containerize** with Docker.
3. **Serve** with FastAPI.
4. **Set up** CI/CD on GitHub Actions to build/push.
5. **Deploy** to AWS (or GCP/Azure) so you have a public endpoint.
6. **Implement** logging and check your logs as you make requests.

**Notes**:
- Keep it small. Even a simple Iris classifier is enough for demonstration.
- Document each step meticulously.
- This project will be a significant piece in your portfolio.

### ADHD Tip
- Break the end-to-end pipeline into micro-steps (Docker build, test locally, push to registry, create ECS service, etc.). Check each off.


## Final Words on Week 5

**Congratulations!** You’ve just covered the **core MLOps pipeline**:
- Docker for containerization.
- Model serving with an API framework.
- CI/CD for automated building & testing.
- Cloud deployment for production hosting.
- Monitoring & logging to keep track of model health.

These steps are **industry must-haves** for any ML engineer or data scientist working in production.

### Coming Next: Week 6
We’ll look at **Large-Scale & Distributed Systems** and how to handle big data and advanced deployment scenarios.

### ADHD Reminder
- Reflect on all you’ve learned. Reward yourself for completing these big tasks.
- Revisit any challenging steps in short, focused sessions. Small sprints + frequent breaks = success.

## End of Week 5 Notebook
---
You’re ready to **ship** ML models like a pro!